# Decision Tree Model

Creating a decision tree model to predict the outcome of an premier league game.

## Importing Libraries

In [2]:
# ML libraries
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Data processing libraries
import pandas as pd

# Visuals
#!pip install graphviz
#!pip install pydotplus
#!pip install IPython
#import graphviz
#import pydotplus
#from IPython.display import Image
from matplotlib import pyplot as plt

## Loading datasets

In [3]:
df = pd.read_csv(r"League_Result_Data/TimeEncoded_PremierLeague_Stats_From_2014to2021.csv")

## Normal Dataset (Time Isn't Included)

### Splitting data

The data with be split into training and test data to ensure the integrity and proformance of the model

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 670 entries, 0 to 669
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             670 non-null    int64  
 1   Season                 670 non-null    object 
 2   Season Encoding        670 non-null    int64  
 3   Date                   670 non-null    object 
 4   YearOfSeason           632 non-null    object 
 5   Time                   670 non-null    object 
 6   Time Encoding          670 non-null    int64  
 7   HomeTeam               670 non-null    object 
 8   HomeTeam Encoding      670 non-null    int64  
 9   AwayTeam               670 non-null    object 
 10  AwayTeam Encoding      670 non-null    int64  
 11  FTHG                   670 non-null    float64
 12  FTAG                   670 non-null    float64
 13  FTR                    670 non-null    object 
 14  FTR Encoding           670 non-null    int64  
 15  HTHG  

## Encodes the YearOfSeason values

In [5]:
df = df.dropna()

seasons = set(df['YearOfSeason'].values)
seasons = list(seasons)

# re-orders the list 
seasons = ['2014/15', '2015/16', '2016/17', '2017/18', '2018/19', '2019/20', '2020/21']
seasons

['2014/15', '2015/16', '2016/17', '2017/18', '2018/19', '2019/20', '2020/21']

In [6]:
df['YearOfSeason Encoding'] = -1

for index, value in df['YearOfSeason'].iteritems():
    
    # 2014/15: 0
    if value == seasons[0]:
        df.at[index, 'YearOfSeason Encoding'] = 0
    
    # 2015/16: 1
    if value == seasons[1]:
        df.at[index, 'YearOfSeason Encoding'] = 1
        
    # 2016/17: 2
    if value == seasons[2]:
        df.at[index, 'YearOfSeason Encoding'] = 2
    
    # 2017/18: 3
    if value == seasons[3]:
        df.at[index, 'YearOfSeason Encoding'] = 3
        
    # 2018/19: 4
    if value == seasons[4]:
        df.at[index, 'YearOfSeason Encoding'] = 4
        
    # 2019/20: 5
    if value == seasons[5]:
        df.at[index, 'YearOfSeason Encoding'] = 5
    
    # 2020/21: 6
    if value == seasons[6]:
        df.at[index, 'YearOfSeason Encoding'] = 6 

In [13]:
"""
# Features -- Drops FTR and any categorical value 
X = df.drop(columns=["HTHG", "HTAG", "HTR Encoding", "Unnamed: 0", "FTHG", "FTAG", "Season", "YearOfSeason", "Date", "HomeTeam", "AwayTeam", "Referee", "FTR", "FTR Encoding", "HTR", "Referee"])
# Labels
y = df["FTR Encoding"]

"""
# Features -- Drops FTR and any categorical value 
X = df.drop(columns=["Unnamed: 0", "Season", "YearOfSeason", "Time","Date", "HomeTeam", "AwayTeam", "Referee", "FTR", "FTR Encoding", "HTR", "Referee"])
# Labels
y = df["FTR Encoding"]

In [14]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 632 entries, 0 to 669
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Season Encoding        632 non-null    int64  
 1   Time Encoding          632 non-null    int64  
 2   HomeTeam Encoding      632 non-null    int64  
 3   AwayTeam Encoding      632 non-null    int64  
 4   FTHG                   632 non-null    float64
 5   FTAG                   632 non-null    float64
 6   HTHG                   632 non-null    float64
 7   HTAG                   632 non-null    float64
 8   Referee Encoding       632 non-null    int64  
 9   Fouls Called Per Game  632 non-null    float64
 10  HS                     632 non-null    float64
 11  AS                     632 non-null    float64
 12  HST                    632 non-null    float64
 13  AST                    632 non-null    float64
 14  HF                     632 non-null    float64
 15  AF    

In [15]:
X

Season Encoding  Time Encoding  HomeTeam Encoding  AwayTeam Encoding  \
0                  0              0                  8                  0   
1                  0              2                  6                 21   
2                  0              2                 13                 11   
3                  0              2                 28                 17   
4                  0              1                 27                 12   
..               ...            ...                ...                ...   
665                3              1                 12                 15   
666                3              1                 14                 18   
667                3              1                 17                 13   
668                3              1                 21                 20   
669                3              1                 28                 -1   

     FTHG  FTAG  HTHG  HTAG  Referee Encoding  Fouls Called Per Game  ...  \
0     0.0   3.0   0.0   1.0                 7                   14.0  ...   
1     1.0   0.0   1.0   0.0                27                   14.0  ...   
2     4.0   3.0   3.0   2.0                28                   14.0  ...   
3     0.0   2.0   0.0   0.0                11                   13.0  ...   
4     0.0   3.0   0.0   0.0                10                   15.0  ...   
..    ...   ...   ...   ...               ...                    ...  ...   
665   0.0   2.0   0.0   0.0                16                   14.0  ...   
666   5.0   0.0   2.0   0.0                25                   15.0  ...   
667   1.0   3.0   1.0   1.0                10                   15.0  ...   
668   3.0   1.0   0.0   1.0                 3                   13.0  ...   
669   1.0   1.0   0.0   0.0                28                   14.0  ...   

     AST    HF    AF   HC   AC   HY   AY   HR   AR  YearOfSeason Encoding  
0    6.0  12.0  12.0  2.0  3.0  2.0  2.0  0.0  0.0                      6  
1    5.0  14.0  11.0  7.0  3.0  2.0  1.0  0.0  0.0                      6  
2    3.0   9.0   6.0  9.0  0.0  1.0  0.0  0.0  0.0                      6  
3    2.0  13.0   7.0  8.0  7.0  2.0  2.0  0.0  0.0                      6  
4    7.0  12.0   9.0  2.0  5.0  1.0  1.0  0.0  0.0                      6  
..   ...   ...   ...  ...  ...  ...  ...  ...  ...                    ...  
665  3.0  12.0  11.0  3.0  3.0  1.0  4.0  1.0  0.0                      5  
666  4.0   7.0   4.0  9.0  0.0  1.0  1.0  0.0  0.0                      5  
667  6.0  11.0   5.0  2.0  4.0  1.0  0.0  0.0  0.0                      5  
668  3.0   9.0  16.0  9.0  1.0  0.0  1.0  0.0  0.0                      5  
669  4.0  16.0  13.0  0.0  7.0  2.0  1.0  0.0  0.0                      5  

[632 rows x 23 columns]

In [16]:
"""
Target Varible - Full Time Results
0 - Home Team Win
1 - Away Team Win
2 - Draw
"""
y

0      1
1      0
2      0
3      1
4      1
      ..
665    1
666    0
667    1
668    0
669    2
Name: FTR Encoding, Length: 632, dtype: int64

In [17]:
# Splitting the data
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [18]:
# Scaling varibles to unit variance
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

## Fitting the model

Fitting the data to a decision tree model

In [19]:
# Declaring the decision tree classifier
tree_clf = tree.DecisionTreeClassifier(max_leaf_nodes = 15, 
                                      max_features=10,
                                      random_state=42,
                                      min_samples_split=3)

# Training the decision tree
clf_train = tree_clf.fit(X_train, y_train)

In [20]:
score = clf_train.score(X_test, y_test)

print(score)

0.9133858267716536


In [21]:
# Confusion Matrix

cv_score = cross_val_score(clf_train, X_train, y_train, cv=3, scoring='accuracy')

print(cv_score, '\n')

y_train_pred = cross_val_predict(clf_train, X_train, y_train, cv=3)
print(confusion_matrix(y_train, y_train_pred))

[0.85798817 0.9702381  0.86309524] 

[[193   1  16]
 [  1 164  12]
 [ 20   2  96]]


In [22]:
# Precision, Recall, F1

print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91       210
           1       0.98      0.93      0.95       177
           2       0.77      0.81      0.79       118

    accuracy                           0.90       505
   macro avg       0.89      0.89      0.89       505
weighted avg       0.90      0.90      0.90       505



## Visualizing the tree

Create visual plots for the decision tree

In [23]:
fn = list(X.columns)
cn ='0', '1', '2'

In [24]:
cn

('0', '1', '2')

In [25]:
tree.plot_tree(clf_train)

[Text(155.98636363636365, 199.32, 'X[5] <= 0.192\ngini = 0.65\nsamples = 505\nvalue = [210, 177, 118]'),
 Text(91.30909090909091, 163.07999999999998, 'X[4] <= 0.048\ngini = 0.559\nsamples = 326\nvalue = [191, 42, 93]'),
 Text(76.0909090909091, 126.83999999999999, 'X[5] <= -0.605\ngini = 0.615\nsamples = 179\nvalue = [44, 42, 93]'),
 Text(30.436363636363637, 90.6, 'X[17] <= -1.206\ngini = 0.488\nsamples = 76\nvalue = [44, 0, 32]'),
 Text(15.218181818181819, 54.359999999999985, 'gini = 0.278\nsamples = 6\nvalue = [1, 0, 5]'),
 Text(45.654545454545456, 54.359999999999985, 'X[4] <= -0.741\ngini = 0.474\nsamples = 70\nvalue = [43, 0, 27]'),
 Text(30.436363636363637, 18.119999999999976, 'gini = 0.0\nsamples = 27\nvalue = [0, 0, 27]'),
 Text(60.872727272727275, 18.119999999999976, 'gini = 0.0\nsamples = 43\nvalue = [43, 0, 0]'),
 Text(121.74545454545455, 90.6, 'X[6] <= -0.209\ngini = 0.483\nsamples = 103\nvalue = [0, 42, 61]'),
 Text(106.52727272727273, 54.359999999999985, 'X[4] <= -0.741\ngi

In [26]:
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (20,30), dpi=300)
tree.plot_tree(
    clf_train,
    feature_names = fn, 
    class_names=cn,           
    filled = True);

# 0 = Home win
# 1 = Away win
# 2 = Draw

Helpful link: https://towardsdatascience.com/scikit-learn-decision-trees-explained-803f3812290d